In [19]:
def read(filename):
    with open(filename, 'r') as f:
        N, K = map(int, f.readline().split())
        d = [0] + list(map(float, f.readline().split()))
        t = np.zeros((N + 1, N + 1))
        for i in range(N + 1):
            t[i] = list(map(float, f.readline().split()))

    return N, K, d, t


import numpy as np

def cal_cost(router, t, d):
    cost = [0 for i in range(len(router))]
    for i in range(len(router)):
        for j in range(len(router[i]) - 1):
            cost[i] += t[router[i][j]][router[i][j + 1]] + d[router[i][j + 1]]
    return cost

def UCV(visited):
        unvisited = []
        for pos in range(len(visited)):
            if visited[pos] == 0:
                unvisited.append(pos)
        return unvisited

def local_search_greedy1(N, K, d, t):
    router = []
    for i in range(K):
        router.append([0])

    visited = np.zeros(N + 1, dtype = int)
    visited[0] = 1      
        
    k = 0
    while sum(visited) != N + 1:
        start = router[k][-1]
        next_pos = 0
        next_time = 1000000
        unvisited = UCV(visited)
        for pos in unvisited:
            if t[start][pos] + d[pos] < next_time:
                next_time = t[start][pos] + d[pos]
                next_pos = pos
        router[k].append(next_pos)
        visited[next_pos] = 1
        k = (k + 1) % K
    
    for i in range(K):
        router[i].append(0)
    cost = [0 for i in range(K)]
    for i in range(K):
        for j in range(len(router[i]) - 1):
            cost[i] += t[router[i][j]][router[i][j + 1]] + d[router[i][j + 1]]
    history_count = 0
    while True:
        max_cost = max(cost)
        max_index = [i for i, j in enumerate(cost) if j == max_cost]
        router_neighborhood = []

        swap1 = np.random.choice(max_index)
        swap2 = np.random.randint(0, K)
        while swap2 == swap1:
            swap2 = np.random.randint(0, K)
        
        if len(router[swap1]) == 3:
            a1 = 1
            b1 = 2
        else:
        #a1 != b1
            a1 = np.random.randint(1, len(router[swap1]) - 2)
            b1 = np.random.randint(a1 + 1, len(router[swap1]) - 1)

        if len(router[swap2]) == 3:
            a2 = 1
            b2 = 2
        else:
            a2 = np.random.randint(1, len(router[swap2]) - 2)
            b2 = np.random.randint(a2 + 1, len(router[swap2]) - 1)

        new_router1 = router[swap1][:a1] + router[swap2][a2:b2] + router[swap1][b1:]
        new_router2 = router[swap2][:a2] + router[swap1][a1:b1] + router[swap2][b2:]

        for i in range(K):
            if i == swap1:
                router_neighborhood.append(new_router1)
            elif i == swap2:
                router_neighborhood.append(new_router2)
            else:
                router_neighborhood.append(router[i])
        
        cost_neighborhood = cal_cost(router_neighborhood, t, d)
        new_max_cost = max(cost_neighborhood)
        if new_max_cost < max_cost:
            router = router_neighborhood
            cost = cost_neighborhood
            history_count = 0
        else:
            history_count += 1
        if history_count == 5000:
            break
        
    return router


In [20]:
def creat_cost(N, d, t):
    cost_matrix = np.zeros((N + 1, N + 1))
    for i in range(N + 1):
        for j in range(N + 1):
            if i != j:
                cost_matrix[i][j] = d[j] + t[i][j]

    return cost_matrix

In [21]:
def local_search_greedy2(N, K, d, t):
    cost_matrix = creat_cost(N, d, t)

    router = [[0] for i in range(K)]
    cost = [0 for i in range(K)]
    tuple_dis_depot = []
    for i in range(1, N + 1):
        tuple_dis_depot.append((i, cost_matrix[0][i]))
    tuple_dis_depot.sort(key = lambda x: x[1])

    while len(tuple_dis_depot) > 0:
        # lay phan tu dau tien trong tuple_dis_depot
        first = tuple_dis_depot[0][0]
        # xoa phan tu dau tien trong tuple_dis_depot
        tuple_dis_depot.pop(0)
        # Them vao router lam tang it chi phi nhat

        cost_value = [cost[i] + cost_matrix[router[i][-1]][first] for i in range(K)]
        min_cost = min(cost_value)

        index_list = [index for index, value in enumerate(cost_value) if value == min_cost]

        router_selected = index_list[0]
        for index in index_list[1:]:
            if cost[index] < cost[router_selected]:
                router_selected = index
        
        router[router_selected].append(first)
        cost[router_selected] += cost_matrix[router[router_selected][-2]][first]

    for i in range(K):
        router[i].append(0)
        cost[i] += cost_matrix[router[i][-2]][0]
    history_count = 0
    while True:
        max_cost = max(cost)
        max_index = [i for i, j in enumerate(cost) if j == max_cost]
        router_neighborhood = []

        swap1 = np.random.choice(max_index)
        swap2 = np.random.randint(0, K)
        while swap2 == swap1:
            swap2 = np.random.randint(0, K)
        
        if len(router[swap1]) == 3:
            a1 = 1
            b1 = 2
        else:
        #a1 != b1
            a1 = np.random.randint(1, len(router[swap1]) - 2)
            b1 = np.random.randint(a1 + 1, len(router[swap1]) - 1)

        if len(router[swap2]) == 3:
            a2 = 1
            b2 = 2
        else:
            a2 = np.random.randint(1, len(router[swap2]) - 2)
            b2 = np.random.randint(a2 + 1, len(router[swap2]) - 1)

        new_router1 = router[swap1][:a1] + router[swap2][a2:b2] + router[swap1][b1:]
        new_router2 = router[swap2][:a2] + router[swap1][a1:b1] + router[swap2][b2:]

        for i in range(K):
            if i == swap1:
                router_neighborhood.append(new_router1)
            elif i == swap2:
                router_neighborhood.append(new_router2)
            else:
                router_neighborhood.append(router[i])
        
        cost_neighborhood = cal_cost(router_neighborhood, t, d)
        new_max_cost = max(cost_neighborhood)
        if new_max_cost < max_cost:
            router = router_neighborhood
            cost = cost_neighborhood
            history_count = 0
        else:
            history_count += 1
        if history_count == 5000:
            break
        
    return router

In [22]:
import numpy as np
import random


def cal_time_employee(segment, d, t):
    if len(segment) == 0:
        return 0
    time_work = 0
    time_work += t[0][segment[0]]
    for i in range(len(segment) - 1):
        time_work += t[segment[i]][segment[i+1]] + d[segment[i]]
    time_work += t[segment[-1]][0] + d[segment[-1]]
    return time_work

def split_segment(chromosome, N):
    segment = []
    segment_employee = []
    for i in range(len(chromosome)):
        if chromosome[i] > N:
            segment.append(segment_employee)
            segment_employee = []
        else:
            segment_employee.append(chromosome[i])
    segment.append(segment_employee)
    return segment
        


class Individual:
    def __init__(self, N, K, d, t):
        self.chromosome = list(np.zeros(N + K - 1, dtype = int))
        self.fitness = None
        self.N = N
        self.K = K
        self.d = d
        self.t = t
    
    def cal_fitness(self):
        segment = split_segment(self.chromosome, self.N)
        max_time = 0
        for i in range (self.K):
            max_time = max(max_time, cal_time_employee(segment[i], self.d, self.t))
        self.fitness = max_time
        
    def gen_random(self):
        number_list = list(range(1, self.N+self.K))
        self.chromosome = list(random.sample(number_list, len(number_list)))
        self.cal_fitness()

    def gen_greedy1(self):
        router = local_search_greedy1(self.N, self.K, self.d, self.t)
        temp = []
        for i in range(self.K):
            if(i == 0):
                temp += router[i][1:-1]
            else:
                temp.append(self.N + i)
                temp += router[i][1:-1]
        # convert to int
        self.chromosome = np.array(temp, dtype= int)
        self.cal_fitness()
    
    def gen_greedy2(self):
        router = local_search_greedy2(self.N, self.K, self.d, self.t)
        temp = []
        for i in range(self.K):
            if(i == 0):
                temp += router[i][1:-1]
            else:
                temp.append(self.N + i)
                temp += router[i][1:-1]
        # convert to int
        self.chromosome = np.array(temp, dtype= int)
        self.cal_fitness()

    
    
    
class Population:
    def __init__(self, N, K, d, t, POP_SIZE = 100, MAX_GEN = 1000, crossover_parameter = 0.9, mutation_parameter = 0.1):
        self.indivs = []
        self.N = N
        self.K = K
        self.d = d
        self.t = t
        self.POP_SIZE = POP_SIZE
        self.MAX_GEN = MAX_GEN
        self.crossover_parameter = crossover_parameter
        self.mutation_parameter = mutation_parameter
    
    def gen_pop(self):
        indi1 = Individual(self.N, self.K, self.d, self.t)
        indi1.gen_greedy1()
        indi2 = Individual(self.N, self.K, self.d, self.t)
        indi2.gen_greedy2()
        self.indivs.append(indi1)
        self.indivs.append(indi2)
        for i in range(self.POP_SIZE -1):
            indi = Individual(self.N, self.K, self.d, self.t)
            indi.gen_random()
            self.indivs.append(indi)

    def order_crossover(self, parent1, parent2):
        indi1 = Individual(self.N, self.K, self.d, self.t)
        indi2 = Individual(self.N, self.K, self.d, self.t)
        
        start = random.randint(0, self.N + self.K - 3)
        end = random.randint(start, self.N + self.K -2)
        for i in range(start, end + 1):
            indi1.chromosome[i] = parent1.chromosome[i]
            indi2.chromosome[i] = parent2.chromosome[i]
        
        indi1_index = end + 1
        indi1_i = 0
        while indi1_index < self.N + self.K - 1:
            if parent2.chromosome[indi1_i] not in indi1.chromosome:
                indi1.chromosome[indi1_index] = parent2.chromosome[indi1_i]
                indi1_index += 1
            indi1_i += 1
        indi1_index = 0
        while indi1_index < start:
            if parent2.chromosome[indi1_i] not in indi1.chromosome:
                indi1.chromosome[indi1_index] = parent2.chromosome[indi1_i]
                indi1_index += 1
            indi1_i += 1
        
        indi2_index = end + 1
        indi2_i = 0
        while indi2_index < self.N + self.K - 1:
            if parent1.chromosome[indi2_i] not in indi2.chromosome:
                indi2.chromosome[indi2_index] = parent1.chromosome[indi2_i]
                indi2_index += 1
            indi2_i += 1
        
        indi2_index = 0
        while indi2_index < start:
            if parent1.chromosome[indi2_i] not in indi2.chromosome:
                indi2.chromosome[indi2_index] = parent1.chromosome[indi2_i]
                indi2_index += 1
            indi2_i += 1
        
        indi1.cal_fitness()
        indi2.cal_fitness()
        return indi1, indi2

    def mutation(self, parent):
        indi = Individual(self.N, self.K, self.d, self.t)
        index1 = random.randint(0, self.N + self.K -2)
        index2 = random.randint(0, self.N + self.K -2)
        while index1 == index2:
            index2 = random.randint(0, self.N + self.K -2)
        
        indi.chromosome = parent.chromosome.copy()
        indi.chromosome[index1], indi.chromosome[index2] = indi.chromosome[index2], indi.chromosome[index1]
        indi.cal_fitness()
        return indi
        
    def evolve(self):
        for i in range(self.MAX_GEN):
            for j in range(self.POP_SIZE):
                if random.random() < self.crossover_parameter:
                    parent1_index = random.randint(0, self.POP_SIZE - 1)
                    parent2_index = random.randint(0, self.POP_SIZE - 1)
                    while parent1_index == parent2_index:
                        parent2_index = random.randint(0, self.POP_SIZE - 1)
                    parent1 = self.indivs[parent1_index]
                    parent2 = self.indivs[parent2_index]
                    
                    indi1, indi2 = self.order_crossover(parent1, parent2)
                    if random.random() < self.mutation_parameter:
                        indi1 = self.mutation(indi1)
                        indi2 = self.mutation(indi2)
                    self.indivs.append(indi1)
                    self.indivs.append(indi2)
            self.indivs.sort(key = lambda x: x.fitness)
            self.indivs = self.indivs[:self.POP_SIZE]
        return self.indivs[0].fitness, self.indivs[0].chromosome


In [23]:
def read_data(file_data):
    data_path = "data\\" + file_data
    with open(data_path, 'r') as f:
        N, K = [int(x) for x in f.readline().split()]
        d = [int(x) for x in f.readline().split()]
        d.insert(0, 0)
        t = []
        for i in range(N + K):
            t.append([int(x) for x in f.readline().split()])
    return N, K, d, t

In [24]:
import time
def ga(file_data):
    N, K, d, t = read_data(file_data)
    time_start = time.time()
    pop = Population(N, K, d, t)
    pop.gen_pop()
    fitness, chromosome = pop.evolve()
    segment = split_segment(chromosome, N)
    time_pro = time.time() - time_start
    result_path = "GA\\1_" + file_data
    with open(result_path, 'w') as f:
        f.write("Objective: " + str(fitness) + '\n')
        f.write("Time: " + str(time_pro) + '\n')
        f.write(str(K) + '\n')
        for se in segment:
            f.write(str(len(se) + 2) + '\n')
            f.write('0 ')
            for pos in se:
                f.write(str(pos) + ' ')
            f.write('0\n')
    print(K)
    for se in segment:
        print(len(se) + 2)
        print(0, end = ' ')
        for pos in se:
            print(pos, end = ' ')
        print(0)
    print(fitness)

In [25]:
data = ["data_5_2.txt", "data_5_3.txt", "data_10_2.txt", "data_10_3.txt","data_10_5.txt", "data_15_3.txt", "data_15_5.txt", "data_15_7.txt", "data_20_5.txt", "data_20_7.txt","data_20_10.txt", "data_50_5.txt", "data_50_10.txt", "data_50_20.txt", "data_100_10.txt", "data_200_10.txt", "data_200_20.txt", "data_400_40.txt", "data_500_50.txt","data_700_70.txt", "data_900_90.txt", "data_1000_100.txt"]

In [26]:
for file_data in data:
    ga(file_data)

2
5
0 4 1 5 0
4
0 3 2 0
360
3
4
0 3 4 0
3
0 5 0
4
0 2 1 0
861
2
7
0 5 8 9 4 7 0
7
0 3 1 10 6 2 0
540
3
5
0 2 1 3 0
6
0 9 7 4 5 0
5
0 8 10 6 0
1929
5
3
0 8 0
5
0 4 5 10 0
4
0 6 2 0
3
0 1 0
5
0 7 3 9 0
1193
3
7
0 1 10 11 13 2 0
6
0 5 3 8 15 0
8
0 9 4 14 12 7 6 0
3640
5
5
0 12 15 1 0
6
0 7 5 9 4 0
5
0 10 3 8 0
5
0 13 11 14 0
4
0 2 6 0
1802
7
4
0 6 12 0
5
0 3 11 2 0
4
0 9 4 0
4
0 5 10 0
4
0 8 13 0
4
0 14 1 0
4
0 15 7 0
1453
5
5
0 7 8 19 0
6
0 14 18 11 6 0
6
0 9 16 17 20 0
7
0 13 3 10 2 1 0
6
0 5 15 4 12 0
2298
7
5
0 11 8 13 0
4
0 9 19 0
5
0 1 6 17 0
5
0 14 10 3 0
4
0 4 16 0
4
0 12 2 0
7
0 20 5 15 18 7 0
1368
10
4
0 3 13 0
4
0 20 6 0
4
0 5 12 0
4
0 4 10 0
4
0 19 7 0
4
0 14 9 0
4
0 2 15 0
4
0 11 17 0
4
0 8 1 0
4
0 16 18 0
1679
5
11
0 20 44 46 39 49 45 26 21 23 0
13
0 12 5 15 34 36 9 19 11 7 10 3 0
11
0 1 41 48 4 18 8 43 47 2 0
13
0 6 27 31 42 25 17 33 38 40 50 13 0
12
0 28 35 22 29 16 30 24 14 32 37 0
6391


KeyboardInterrupt: 